## Import libraries. Load dataset from ArenaStats-TBC.

In [26]:
import pandas as pd
import datetime
pd.options.display.max_columns = 7

In [27]:
arena = pd.read_csv("C:\\Users\\chris\\Desktop\\arena.csv")
icons = pd.read_csv("C:\\Users\\chris\\Documents\\Datasets\\WOW\\WOW_CLASS_ICONS.csv")

In [28]:
class_icons = {}
for index,row in icons.iterrows():
    class_icons[row["class"]] = row["class_icon"]
class_icons[""] = ""
class_icons

{'DRUID': 'https://www.progamerreview.com/wp-content/uploads/2019/07/E0FwJT8.png',
 'HUNTER': 'https://www.progamerreview.com/wp-content/uploads/2019/07/ybbMpYe.png',
 'MAGE': 'https://www.progamerreview.com/wp-content/uploads/2019/07/IsuHHZo.png',
 'PALADIN': 'https://www.progamerreview.com/wp-content/uploads/2019/07/LxbAt4V.png',
 'PRIEST': 'https://www.progamerreview.com/wp-content/uploads/2019/07/RCXyIlL.png',
 'ROGUE': 'https://www.progamerreview.com/wp-content/uploads/2019/07/90odTOu.png',
 'SHAMAN': 'https://www.progamerreview.com/wp-content/uploads/2019/07/q4r80Pn.png',
 'WARLOCK': 'https://www.progamerreview.com/wp-content/uploads/2019/07/PDrI6st.png',
 'WARRIOR': 'https://www.progamerreview.com/wp-content/uploads/2019/07/duY4PXM.png',
 '': ''}

## Append new columns (queueType, playersPerTeam).

In [29]:
queue_type_col = []
players_per_team = []
for index,row in arena.iterrows():
    player_count = 0
    for i in ["teamPlayerClass1","teamPlayerClass2","teamPlayerClass3","teamPlayerClass4","teamPlayerClass5"]:
        if str(row[i]) == "nan":
            break
        else:
            player_count += 1
    queue_type_col.append(f"{player_count}v{player_count}")
    players_per_team.append(player_count)
arena["queueType"] = queue_type_col
arena["playersPerTeam"] = players_per_team
arena.fillna("",inplace = True)

## Dataset cleaning functions. (To fix team faction/player/match errors)

In [30]:
def faction_count(players):
    horde_races = ["SCOURGE","TROLL","TAUREN","BLOODELF","ORC"]
    ally_races = ["HUMAN","GNOME","DRAENEI","NIGHTELF","DWARF"]
    ally = 0
    horde = 0
    for i in players:
        if (i[2] in horde_races):
            horde += 1
        elif (i[2] in ally_races):
            ally += 1
    return {"HORDE" : horde, "ALLIANCE" : ally}

def players_by_faction(players):
    horde_races = ["SCOURGE","TROLL","TAUREN","BLOODELF","ORC"]
    ally_races = ["HUMAN","GNOME","DRAENEI","NIGHTELF","DWARF"]
    players_horde = []
    players_ally = []
    for i in players:
        if (i[2] in horde_races):
            players_horde.append(i)
        elif (i[2] in ally_races):
            players_ally.append(i)
    return {"HORDE" : players_horde,"ALLIANCE" : players_ally}

def fix_players(players, players_prev):
    if (faction_count(players)["HORDE"] == row["playersPerTeam"]) and (faction_count(players)["ALLIANCE"] != row["playersPerTeam"]):
        return players_by_faction(players)["HORDE"]
    elif (faction_count(players)["HORDE"] != row["playersPerTeam"]) and (faction_count(players)["ALLIANCE"] == row["playersPerTeam"]):
        return players_by_faction(players)["ALLIANCE"]

    players_rev = []
    players_removed = []
    for cur_index,cur_i in enumerate(players):
        match = False
        for prev_index,prev_i in enumerate(players_prev):
            if (cur_i[1] == prev_i[1]) and (cur_i[2] == prev_i[2]):
                players_prev.pop(prev_index)
                match = True
                break
        if match == False:
            players_rev.append(cur_i)
    return players_rev

def get_faction(players):
    horde_races = ["SCOURGE","TROLL","TAUREN","BLOODELF","ORC"]
    ally_races = ["HUMAN","GNOME","DRAENEI","NIGHTELF","DWARF"]
    horde = False
    ally = False
    for i in players:
        if i[2] in horde_races:
            horde = True
        elif i[2] in ally_races:
            ally = True
    if (horde and ally) or (not horde and not ally):
        return "NA"
    elif horde:
        return "HORDE"
    elif ally:
        return "ALLIANCE"

## Drop duplicated games and skirmishes. Fix "ghost player" errors. Format map name (zoneId) and startTime. Update team factions. Drop rows with unfixable errors.

In [31]:
rows_to_drop = []
for index,row in arena.iterrows():
    # Duplicated games error.
    if row["duration"] == 0:
        rows_to_drop.append(index)
        continue
    # Omit skirmishes.
    if row["isRanked"] != "YES":
        rows_to_drop.append(index)
        continue
    if row["teamName"] == "":
        rows_to_drop.append(index)
        continue
    players = []
    for i in range(1,6):
        if row[f"enemyPlayerClass{i}"] == "":
            break
        else:
            players.append((row[f"enemyPlayerName{i}"],row[f"enemyPlayerClass{i}"],row[f"enemyPlayerRace{i}"]))
    arena.loc[index, "enemyFaction"] = get_faction(players)
    #Ghost player error.
    if len(players) > row["playersPerTeam"]:
        if index == 0:
            rows_to_drop.append(index)
            continue
        players_rev = fix_players(players, players_prev)
        if len(players_rev) != row["playersPerTeam"]:
            rows_to_drop.append(index)
        else:
            arena.loc[index, "enemyFaction"] = get_faction(players_rev)
            for i in range(1,6):
                arena.loc[index, f"enemyPlayerName{i}"] = ""
                arena.loc[index, f"enemyPlayerClass{i}"] = ""
                arena.loc[index, f"enemyPlayerRace{i}"] = ""
            for index_rev,i in enumerate(players_rev):
                arena.loc[index, f"enemyPlayerName{index_rev + 1}"] = i[0]
                arena.loc[index, f"enemyPlayerClass{index_rev + 1}"] = i[1]
                arena.loc[index, f"enemyPlayerRace{index_rev + 1}"] = i[2]
    # Replace zoneId with map names.
    if row["zoneId"] == 3698:
        arena.loc[index, "zoneId"] = "Blade's Edge Arena"
    elif row["zoneId"] == 3702:
        arena.loc[index, "zoneId"] = "Nagrand Arena"
    elif row["zoneId"] == 3968:
        arena.loc[index, "zoneId"] = "Ruins of Lordaeron"
    arena.loc[index, "startTime"] = datetime.datetime.fromtimestamp(row["startTime"]).strftime("%Y-%m-%d-%H:%M")
    players_prev = players
arena.drop(rows_to_drop, inplace=True)
arena.drop("endTime", axis=1, inplace=True)

## Append more columns (teamComp, enemyTeamComp, winLoss).

In [32]:
classes = ["DRUID","HUNTER","MAGE","PALADIN","PRIEST","ROGUE","SHAMAN","WARLOCK","WARRIOR"]
team_comp_dict = {}
enemy_team_comp_dict = {}
team_comp = ""
enemy_team_comp = ""
tc_list = []
etc_list = []
win_loss_list = []

# Initialize team comp dicts.
for i in classes:
    team_comp_dict[i] = 0
    enemy_team_comp_dict[i] = 0
for index,row in arena.iterrows():
    team_comp = ""
    enemy_team_comp = ""
    for i in range(1,6):
        if row[f"teamPlayerClass{i}"] in classes:
            team_comp_dict[row[f"teamPlayerClass{i}"]] += 1
        if row[f"enemyPlayerClass{i}"] in classes:
            enemy_team_comp_dict[row[f"enemyPlayerClass{i}"]] += 1
    for i in team_comp_dict:
        if team_comp_dict[i] == 0:
            continue
        while team_comp_dict[i] > 0:
            if team_comp != "":
                team_comp += "-"
            team_comp += i
            team_comp_dict[i] -= 1
    for i in enemy_team_comp_dict:
        if enemy_team_comp_dict[i] == 0:
            continue
        while enemy_team_comp_dict[i] > 0:
            if enemy_team_comp != "":
                enemy_team_comp += "-"
            enemy_team_comp += i
            enemy_team_comp_dict[i] -= 1
    if row["diffRating"] == "":
        win_loss_list.append("NA")
    elif int(row["diffRating"]) > 0:
        win_loss_list.append("1")
    else:
        win_loss_list.append("0")
    tc_list.append(team_comp)
    etc_list.append(enemy_team_comp)
arena["teamComp"] = tc_list
arena["enemyTeamComp"] = etc_list
arena["winLoss"] = win_loss_list

## Append columns for class icons.

In [33]:
arena.reset_index(inplace=True)

In [34]:
teamClassIcon = []
enemyTeamClassIcon = []

for index,row in arena.iterrows():
    teamClassIconRow = ["","","","",""]
    enemyTeamClassIconRow = ["","","","",""]
    for index,i in enumerate(row["teamComp"].split("-")):
        teamClassIconRow[index] = (class_icons[i])
    for index,i in enumerate(row["enemyTeamComp"].split("-")):
        enemyTeamClassIconRow[index] = (class_icons[i])
    teamClassIcon.append(teamClassIconRow)
    enemyTeamClassIcon.append(enemyTeamClassIconRow)

df_icons1 = pd.DataFrame(teamClassIcon, columns = ["teamClassIcon1",
                                                   "teamClassIcon2",
                                                   "teamClassIcon3",
                                                   "teamClassIcon4",
                                                   "teamClassIcon5"])
df_icons2 = pd.DataFrame(enemyTeamClassIcon, columns = ["enemyTeamClassIcon1",
                                                        "enemyTeamClassIcon2",
                                                        "enemyTeamClassIcon3",
                                                        "enemyTeamClassIcon4",
                                                        "enemyTeamClassIcon5"])
arena = pd.concat([arena, df_icons1, df_icons2], axis=1)

## Save and export cleaned dataset.

In [35]:
arena.head()

,index,isRanked,startTime,...,enemyTeamClassIcon3,enemyTeamClassIcon4,enemyTeamClassIcon5
0,0,YES,2022-01-28-14:27,...,,,
1,1,YES,2022-01-28-14:31,...,,,
2,2,YES,2022-01-28-14:35,...,,,
3,3,YES,2022-01-28-14:38,...,,,
4,4,YES,2022-01-28-14:42,...,,,


In [36]:
arena.to_csv("C:\\Users\\chris\\Desktop\\arena_clean.csv", index=False)